<a href="https://colab.research.google.com/github/NicolaGabriele/progettoDeepLearning/blob/main/ProgettoDeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.9 MB/s eta 0:00:00


In [69]:
import cv2
import torch
import numpy as np
import torch.nn as nn
from PIL import Image
import matplotlib.pyplot as plt
from torchsummary import summary
from sklearn.metrics import f1_score
import torchvision.transforms as transforms
from torchvision import models as resnet_model
import torchvision
from torcheval.metrics import BinaryF1Score
import os
from torch.utils.data.dataset import random_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(str(device) + ' is selected !')

IMAGES_DIR = '/content/drive/MyDrive/machinedeeplearning/BUS_UC/All/images'
LABELS_DIR = '/content/drive/MyDrive/machinedeeplearning/BUS_UC/All/masks'

cuda is selected !


#***Dataset Loading***

In [88]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, img_dir, lab_dir, train = True):
    super(Dataset, self).__init__()
    self.img_dir = img_dir
    self.lab_dir = lab_dir
    self.img_names = os.listdir(img_dir)
    self.lab_names = os.listdir(lab_dir)
    self.transform1 = torchvision.transforms.Compose(
        [torchvision.transforms.Resize((224,224)),
        torchvision.transforms.RandomHorizontalFlip(p=0.5),
        torchvision.transforms.RandomVerticalFlip(p=0.5),
        torchvision.transforms.GaussianBlur(3)]
    )
    self.transform = torchvision.transforms.Resize((224,224))
    self.train = train

  def __getitem__(self,idx):
    img_name = os.path.join(self.img_dir,self.img_names[idx])
    mask_name = os.path.join(self.lab_dir, self.lab_names[idx])
    image = torchvision.io.read_image(img_name)/255
    mask = torchvision.io.read_image(mask_name)/255
    if self.train:
      image = self.transform1(image)
      mask = self.transform1(mask)
    else:
      image = self.transform(image)
      mask = self.transform(mask)
    return (image,mask[0])

  def __len__(self):
    return len(os.listdir(self.img_dir))

In [89]:
data = Dataset(IMAGES_DIR,LABELS_DIR,train=False)
validationset, testset, trainset = random_split(data, [0.15,0.15,0.7])

#***MET-NET***

In [7]:
class DecoderBottleneckLayer(nn.Module):
    def __init__(self, in_channels, n_filters, use_transpose=True):
        super(DecoderBottleneckLayer, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, in_channels // 4, 1)
        self.norm1 = nn.BatchNorm2d(in_channels // 4)
        self.relu1 = nn.LeakyReLU()

        if use_transpose:
            self.up = nn.Sequential(
                nn.ConvTranspose2d(
                    in_channels // 4, in_channels // 4, 3, stride=2, padding=1, output_padding=1
                ),
                nn.BatchNorm2d(in_channels // 4),
                nn.LeakyReLU()
            )
        else:
            self.up = nn.Upsample(scale_factor=2, align_corners=True, mode="bilinear")

        self.conv3 = nn.Conv2d(in_channels // 4, n_filters, 1)
        self.norm3 = nn.BatchNorm2d(n_filters)
        self.relu3 = nn.LeakyReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)
        x = self.up(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = self.relu3(x)
        return x

class FFBlock(nn.Module):
    def __init__(self, channels):
        super(FFBlock, self).__init__()

        self.conv3 = nn.Conv2d(in_channels=channels, out_channels=channels, kernel_size=3, padding=1)
        self.conv1 = nn.Conv2d(in_channels=channels, out_channels=channels, kernel_size=1)

        self.relu3 = nn.LeakyReLU()
        self.relu1 = nn.LeakyReLU()

    def forward(self, x):
        x3 = self.conv3(x)
        x3 = self.relu3(x3)
        x1 = self.conv1(x)
        x1 = self.relu1(x1)
        out = x3 + x1

        return out

class SEBlock(nn.Module):
    def __init__(self, channel, r=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // r, bias=False),
            nn.LeakyReLU(),
            nn.Linear(channel // r, channel, bias=False),
            nn.Sigmoid(),
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        ## Squeeze operation
        y = self.avg_pool(x).view(b, c)
        ## Excitation operation
        y = self.fc(y).view(b, c, 1, 1)
        ## Fusion operation
        y = torch.mul(x, y)
        return y

class PDFBlock(nn.Module):
    def __init__(self,in_channels, out_channels_list, kernel_size_list, dilation_list):
        super(PDFBlock, self).__init__()
        self.conv_num = len(out_channels_list)
        assert(self.conv_num == 4)
        assert(self.conv_num == len(kernel_size_list) and self.conv_num == len(dilation_list))
        pad0 = int((kernel_size_list[0] - 1) / 2 * dilation_list[0])
        pad1 = int((kernel_size_list[1] - 1) / 2 * dilation_list[1])
        pad2 = int((kernel_size_list[2] - 1) / 2 * dilation_list[2])
        pad3 = int((kernel_size_list[3] - 1) / 2 * dilation_list[3])
        self.conv_1 = nn.Conv2d(in_channels, out_channels_list[0], kernel_size = kernel_size_list[0], dilation = dilation_list[0], padding = pad0 )
        self.conv_2 = nn.Conv2d(in_channels, out_channels_list[1], kernel_size = kernel_size_list[1], dilation = dilation_list[1], padding = pad1 )
        self.conv_3 = nn.Conv2d(in_channels, out_channels_list[2], kernel_size = kernel_size_list[2], dilation = dilation_list[2], padding = pad2 )
        self.conv_4 = nn.Conv2d(in_channels, out_channels_list[3], kernel_size = kernel_size_list[3], dilation = dilation_list[3], padding = pad3 )

        out_channels  = out_channels_list[0] + out_channels_list[1] + out_channels_list[2] + out_channels_list[3]
        self.conv_1x1 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, kernel_size=1, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU())

    def forward(self, x):
        x1 = self.conv_1(x)
        x2 = self.conv_2(x)
        x3 = self.conv_3(x)
        x4 = self.conv_4(x)

        y  = torch.cat([x1, x2, x3, x4], dim=1)
        y  = self.conv_1x1(y)
        return y

class MET_Net(nn.Module):
    def __init__(self, n_channels=3, n_classes=1):
        super(MET_Net, self).__init__()

        transformer = torch.hub.load('facebookresearch/deit:main', 'deit_tiny_distilled_patch16_224', pretrained=True)
        resnet = resnet_model.resnet18(pretrained=True)

        self.firstconv = resnet.conv1
        self.firstbn = resnet.bn1
        self.firstrelu = resnet.relu

        self.encoder1 = resnet.layer1
        self.encoder2 = resnet.layer2
        self.encoder3 = resnet.layer3
        self.encoder4 = resnet.layer4

        self.patch_embed = transformer.patch_embed
        self.transformers = nn.ModuleList( [transformer.blocks[i] for i in range(12)] )

        self.conv_seq_img = nn.Conv2d(in_channels=192, out_channels=512, kernel_size=1, padding=0)
        self.se = SEBlock(channel=1024)
        self.conv2d = nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1, padding=0)

        self.FFBlock1 = FFBlock(channels=64)
        self.FFBlock2 = FFBlock(channels=128)
        self.FFBlock3 = FFBlock(channels=256)

        self.FFB1 = nn.ModuleList([self.FFBlock1 for i in range(6)])
        self.FFB2 = nn.ModuleList([self.FFBlock2 for i in range(4)])
        self.FFB3 = nn.ModuleList([self.FFBlock3 for i in range(2)])

        filters = [64, 128, 256, 512]

        self.decoder4 = DecoderBottleneckLayer(filters[3], filters[2])
        self.decoder3 = DecoderBottleneckLayer(filters[2], filters[1])
        self.decoder2 = DecoderBottleneckLayer(filters[1], filters[0])
        self.decoder1 = DecoderBottleneckLayer(filters[0], filters[0])

        self.final_conv1 = nn.ConvTranspose2d(filters[0], 32, 4, 2, 1)
        self.final_relu1 = nn.LeakyReLU()
        self.final_conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.final_relu2 = nn.LeakyReLU()
        self.final_conv3 = nn.Conv2d(32, n_classes, 3, padding=1)

    def forward(self, x):
        b, c, h, w = x.shape

        e0 = self.firstconv(x)
        e0 = self.firstbn(e0)
        e0 = self.firstrelu(e0)

        e1 = self.encoder1(e0)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        feature_cnn = self.encoder4(e3)

        emb = self.patch_embed(x)
        for i in range(12):
            emb = self.transformers[i](emb)

        feature_tf = emb.permute(0, 2, 1)
        feature_tf = feature_tf.view(b, 192, 14, 14)
        feature_tf = self.conv_seq_img(feature_tf)

        feature_cat = torch.cat((feature_cnn, feature_tf), dim=1)
        feature_att = self.se(feature_cat)
        feature_out = self.conv2d(feature_att)

        for i in range(2):
            e3 = self.FFB3[i](e3)
        for i in range(4):
            e2 = self.FFB2[i](e2)
        for i in range(6):
            e1 = self.FFB1[i](e1)

        d4 = self.decoder4(feature_out) + e3
        d3 = self.decoder3(d4) + e2
        d2 = self.decoder2(d3) + e1

        out1 = self.final_conv1(d2)
        out1 = self.final_relu1(out1)

        out = self.final_conv2(out1)
        out = self.final_relu2(out)
        out = self.final_conv3(out)

        return out

In [94]:
model = MET_Net()

Using cache found in /root/.cache/torch/hub/facebookresearch_deit_main
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


#***Training***

In [14]:
class DiceLoss(nn.Module):
  def __init__(self):
    super(DiceLoss,self).__init__()

  def forward(self, predictions, targets):
    pred = torch.nn.functional.sigmoid(predictions)
    pred = torch.where(pred>=0.7, 1, 0).squeeze(dim=1)
    targ = targets.squeeze(dim=1)
    s1 = (targ*pred).flatten().sum()
    s2 = (pred**2).flatten().sum()
    s3 = (targ**2).flatten().sum()
    rap = (1+2*s1)/(1+s3+s2)
    return torch.tensor(1-rap.mean(), requires_grad=True)

In [46]:
class BinaryDiceLoss(nn.Module):

    def __init__(self, smooth=1, p=2, reduction='mean'):
        super(BinaryDiceLoss, self).__init__()
        self.smooth = smooth
        self.p = p
        self.reduction = reduction

    def forward(self, predict, target):
        assert predict.shape[0] == target.shape[0], "predict & target batch size don't match"
        predict = predict.contiguous().view(predict.shape[0], -1)
        target = target.contiguous().view(target.shape[0], -1)

        num = torch.sum(torch.mul(predict, target), dim=1) + self.smooth
        den = torch.sum(predict.pow(self.p) + target.pow(self.p), dim=1) + self.smooth

        loss = torch.tensor(1 - num / den, requires_grad=True)

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        elif self.reduction == 'none':
            return loss
        else:
            raise Exception('Unexpected reduction {}'.format(self.reduction))

In [113]:
#iperparameter
num_epochs = 100
batch_size = 4
learning_rate=1e-4
optimizer = torch.optim.NAdam(params=model.parameters(),lr=learning_rate)
lr = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,patience=15)
dataloader = torch.utils.data.DataLoader(trainset,batch_size=batch_size,shuffle=True)
loss_fn = torch.nn.BCEWithLogitsLoss()#BinaryDiceLoss()

In [96]:
def train_one_epoch(epoch_index):
    l = []

    for i, data in enumerate(dataloader):

        inputs, labels = data
        inputs = inputs.to('cuda')
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs).squeeze(dim=1)
        #outputs = torch.nn.functional.sigmoid(outputs)
        loss = loss_fn(outputs, labels)
        loss.backward()

        optimizer.step()
        #print('Batch {} loss: {:.4f}'.format(i,loss.item()))

        l.append(loss.item())

    print('Epoch {} Loss (mean): {:.4f}'.format(epoch_index, np.mean(l)))

In [114]:
model.train()
model.to('cuda')
for epoch in range(61,num_epochs+1):
  train_one_epoch(epoch)
  if epoch%10 == 0:
    torch.save(model,f'/content/drive/MyDrive/machinedeeplearning/models/model_{epoch}ep')

Epoch 61 Loss (mean): 0.0191
Epoch 62 Loss (mean): 0.0286
Epoch 63 Loss (mean): 0.0202
Epoch 64 Loss (mean): 0.0160
Epoch 65 Loss (mean): 0.0146
Epoch 66 Loss (mean): 0.0143
Epoch 67 Loss (mean): 0.0139
Epoch 68 Loss (mean): 0.0137
Epoch 69 Loss (mean): 0.0134
Epoch 70 Loss (mean): 0.0134
Epoch 71 Loss (mean): 0.0128
Epoch 72 Loss (mean): 0.0121
Epoch 73 Loss (mean): 0.0119
Epoch 74 Loss (mean): 0.0623
Epoch 75 Loss (mean): 0.0335
Epoch 76 Loss (mean): 0.0213
Epoch 77 Loss (mean): 0.0148
Epoch 78 Loss (mean): 0.0121
Epoch 79 Loss (mean): 0.0106
Epoch 80 Loss (mean): 0.0101
Epoch 81 Loss (mean): 0.0097
Epoch 82 Loss (mean): 0.0094
Epoch 83 Loss (mean): 0.0092
Epoch 84 Loss (mean): 0.0091
Epoch 85 Loss (mean): 0.0092
Epoch 86 Loss (mean): 0.0090
Epoch 87 Loss (mean): 0.0089
Epoch 88 Loss (mean): 0.0089
Epoch 89 Loss (mean): 0.0089
Epoch 90 Loss (mean): 0.0088
Epoch 91 Loss (mean): 0.0089
Epoch 92 Loss (mean): 0.0089
Epoch 93 Loss (mean): 0.0088
Epoch 94 Loss (mean): 0.0087
Epoch 95 Loss 

In [ ]:
model.to(device)
train_one_epoch(1)

Epoch 1 Loss (mean): 0.4986859874197548


In [ ]:
k = torch.randn((3,3,224,224))
model.to('cpu')
k = model(k)
k = torch.nn.functional.sigmoid(k)
k = torch.where(k>=0.6,1,0)
l = torch.rand((3,1,224,224))
l = torch.where(l>=0.6,1,0)
loss_fn(k,l)

tensor(0.5891)

#***Model Evaluation***

In [132]:
model = torch.load('/content/drive/MyDrive/machinedeeplearning/models/model_100ep')
model.eval()
testloader = torch.utils.data.DataLoader(testset,batch_size=1,shuffle=False)
m = BinaryF1Score(threshold=0.5)
prec = torcheval.metrics.BinaryPrecision(threshold=0.5)
rec = torcheval.metrics.BinaryRecall(threshold=0.5)
for i, data in enumerate(testloader):
  x, ytrue = data
  x = x.to(device)
  ypred = model(x)
  ypred = torch.nn.functional.sigmoid(ypred)[0][0].flatten()
  m.update(ypred,ytrue[0].flatten())
  prec.update(ypred,ytrue[0].flatten())

print(m.compute())
print(prec.compute())


tensor(0.5141)
tensor(0.5678)


In [71]:
import torcheval
ypred = trainset[0][1]
ypred = ypred.flatten()
m = BinaryF1Score(threshold=0.7)
m.update(ypred,ypred)
m.compute()

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


tensor(0.9814)

In [63]:
!pip install torcheval

In [14]:
ytrue = ytrue.to('cpu').detach()
ypred = ypred.squeeze(dim=1).to('cpu').detach()
ypred = torch.nn.functional.sigmoid(ypred)
ypred = torch.where(ypred>=0.7,1,0)
f1_score(y_true=ytrue,y_pred=ypred)

ValueError: unknown is not supported